In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core.chat_engine import CondensePlusContextChatEngine

# 環境変数の取得
load_dotenv("../.env")
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
# ドキュメントからテキスト情報を読込
documents = SimpleDirectoryReader("../data/text").load_data()

# インデックスの構築
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# Chat Engineの作成（ChatMode.OPENAI/REACTが削除されたため代替）
llm = OpenAI(model=MODEL_NAME)

# Retriever（上位件数はデフォルトに寄せたい場合は省略でもOK）
retriever = index.as_retriever(similarity_top_k=2)

# CondensePlusContextChatEngine を使用
chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    llm=llm,
    verbose=True,
)

In [ ]:
# 質問：1回目
response = chat_engine.chat("有給休暇はいつから取得できますか？")

# 言語モデルからの回答を表示
print(response)

In [ ]:
# 引用元を表示（source_nodes を確認）
if hasattr(response, "source_nodes") and response.source_nodes:
    for sn in response.source_nodes:
        md = sn.node.metadata
        print("ファイル名：", md.get("file_name"))
        print("関連度スコア:", sn.score)
        print("テキスト：")
        print(sn.node.text)
        print("-" * 50)
else:
    print("引用元が見つかりませんでした。")

In [ ]:
# 質問：2回目
response = chat_engine.chat("勤続年数が5年の場合は何日ですか？")

# 言語モデルからの回答を表示
print(response)